<a href="https://colab.research.google.com/github/cokoroafor11/Playlist-Recommender-App/blob/main/Spotify_Playlist_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!pip install spotipy --upgrade
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 7.2 MB/s eta 0:00:00


In [2]:
#Things to call:
#Excel sheet with spotify playlists
playlist_sheet = 'Spotify Playlists 1 per genre.xlsx'

## Create the Spotify object

In [3]:
#Save client credentials
client_id = "c1f74565be774e65aa211462aaf5fed8"
client_secret = "2edce4052f8f46639c0e112658572d66"
#Create object
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,client_secret=client_secret),requests_timeout=100,retries=3)

## Functions I Need

### Extract songs from playlist

In [4]:
def extract_songs(playlist):
    '''Function to extract songs from a playlist: Will run this for all the playlists being added'''

    #Make sure playlist input is string
    if type(playlist) != str: 
        playlist = str(playlist)

    #Get playlist tracks    
    playlist_tracks =  spotify.playlist_tracks(playlist)
    tracks = []

    #Append tracks to list if no type error
    for elem in playlist_tracks['items']:
        try:
            tracks.append(elem['track']['external_urls']['spotify'])
        except TypeError:
            pass
    return tracks
  

In [5]:
tracks = extract_songs('https://open.spotify.com/playlist/37i9dQZEVXbqgPCgwknZJW?si=ae7b2ccfd8d64357')

In [6]:
tracks

['https://open.spotify.com/track/40EXFgNMOIQRQOwJZ8mJkC',
 'https://open.spotify.com/track/2630Nsy5LsKK3lbT0pOmI2',
 'https://open.spotify.com/track/7lP071DicbJJgF3VU8vCkw',
 'https://open.spotify.com/track/7dnixpDdQlIOX1L7O0pdFM',
 'https://open.spotify.com/track/2HRa9MR7Sqy3T6WbdpF3Rj',
 'https://open.spotify.com/track/4wqBRSqEPrL4UjP00lZB1M',
 'https://open.spotify.com/track/7FPEPfrrJlVIHk4pGWtAqy',
 'https://open.spotify.com/track/0nTgtjFQzLhvU3VLciMPlN',
 'https://open.spotify.com/track/043bwUOxMoxTIPvjkvHAwS',
 'https://open.spotify.com/track/0cnh6R6nuYgvCaOev5j8UT',
 'https://open.spotify.com/track/16OxpGrrxgyG724pBPIILx',
 'https://open.spotify.com/track/1h6w4POXM2LfG4ZD4xUfZQ',
 'https://open.spotify.com/track/1mXJlLzoEk3STrioO2ehRE',
 'https://open.spotify.com/track/36rwKN7gg5MsyQhtpdxYEi',
 'https://open.spotify.com/track/4i8qIot5wFptu30dXnBx9L',
 'https://open.spotify.com/track/56Gdp4KNxEE8owRsZvAPuC',
 'https://open.spotify.com/track/5edhwujeOj54zlfAPa30C8',
 'https://open

## Get the spotify 

In [7]:
def get_track_info(track):
  track = spotify.track(track)
  track_info = {}
  artists= []
  track_info['name'] = track['name']
  for artist in track['artists']:
        artists.append(artist['name'])
  track_info['artists'] = artists
  track_info['popularity'] = track['popularity']
  return track_info

In [8]:
hi = get_track_info('https://open.spotify.com/track/34YfyXjr42nealgrGr7Zll')

In [9]:
hi['popularity']

59

### Get Artists Names

In [10]:
def get_track_artists(track):
    '''Get all artists for each track'''
    
    artists = []
    
    #Iterate through all avaiable artists, append to new array
    for artist in spotify.track(track)['artists']:
        artists.append(artist['name'])
    return artists

In [11]:
get_track_artists('https://open.spotify.com/track/5OSGdSxtulzlzdPdK2MQJC?si=a4a9db4cd27242b2')

['Marshmello', 'Farruko']

### Get Track Name

In [12]:
def get_track_name(track):
    '''Get track name'''

    return spotify.track(track)['name']

### Get Audio Features

In [13]:
def get_audio_features(song):
    '''Function to get score for feature type of a song'''
    #Get audio features if no type error, otherwise skip
    features = spotify.audio_features(song)[0]
    titles = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
    for elem in titles:
      try:
        features[elem]
      except TypeError:
        features[elem] == None
        pass
      return features

In [14]:
get_audio_features('https://open.spotify.com/track/5OSGdSxtulzlzdPdK2MQJC?si=a4a9db4cd27242b2')

{'danceability': 0.755,
 'energy': 0.676,
 'key': 5,
 'loudness': -7.349,
 'mode': 1,
 'speechiness': 0.0377,
 'acousticness': 0.0418,
 'instrumentalness': 0.00176,
 'liveness': 0.204,
 'valence': 0.461,
 'tempo': 123.98,
 'type': 'audio_features',
 'id': '5OSGdSxtulzlzdPdK2MQJC',
 'uri': 'spotify:track:5OSGdSxtulzlzdPdK2MQJC',
 'track_href': 'https://api.spotify.com/v1/tracks/5OSGdSxtulzlzdPdK2MQJC',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5OSGdSxtulzlzdPdK2MQJC',
 'duration_ms': 209136,
 'time_signature': 4}

What do I need from each song?
1. Artist names
2. Album
3. Genres
4. Song features


In [15]:
spotify.audio_features("https://open.spotify.com/track/5xP9lQYA8YQmQh6BOxcAnR")[0]

{'danceability': 0.852,
 'energy': 0.675,
 'key': 1,
 'loudness': -6.271,
 'mode': 1,
 'speechiness': 0.197,
 'acousticness': 0.0652,
 'instrumentalness': 3.74e-05,
 'liveness': 0.36,
 'valence': 0.83,
 'tempo': 99.012,
 'type': 'audio_features',
 'id': '5xP9lQYA8YQmQh6BOxcAnR',
 'uri': 'spotify:track:5xP9lQYA8YQmQh6BOxcAnR',
 'track_href': 'https://api.spotify.com/v1/tracks/5xP9lQYA8YQmQh6BOxcAnR',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5xP9lQYA8YQmQh6BOxcAnR',
 'duration_ms': 215467,
 'time_signature': 4}

In [16]:
#https://open.spotify.com/playlist/7IKIjt9wZPPQWgvHo1SeFO?si=75371c3c8eaa49a2

In [17]:
songs = extract_songs('https://open.spotify.com/playlist/7IKIjt9wZPPQWgvHo1SeFO?si=75371c3c8eaa49a2')

In [18]:
#Spotify Playlists 1 per genre.xlsx

In [19]:
def excel_list_to_df():
    '''Create the dataframe from the excel sheet of playlists'''

    df = pd.read_excel("Spotify Playlists 1 per genre.xlsx")
    playlists = df['Link']
    return playlists

Let's have a function that will show stats for the playlist

## Build Playlist Database

In [20]:
def populate_song_info(playlist):
    '''
    Function that populates an dictionary where keys are the audio features and values are an array of scores for each song
    This function performs the lionshare of data setup
    '''
    songlist = extract_songs(playlist)

    #Storage for various types of data
    song_data = {}
    name_array = []
    artist_array = []
    popularity_array = []
    dance_array = []
    energy_array = []
    key_array = []
    loudness_array = []
    mode_array = []
    speech_array = []
    acoustic_array = []
    instrument_array = []
    live_array = []
    valence_array = []
    tempo_array = []
    link_array = []
    
    #Loop to append song info to appropriate array
    for song in songlist:
        
        features = spotify.audio_features(song)[0]
        #Check if features don't exist, skip the song entry
        if features ==  None:
          continue
        
        #Get song names, artists, popularity
        track = get_track_info(song)
        name_array.append(track['name'])
        artist_array.append(track['artists'])
        popularity_array.append(track['popularity'])

        #Append feature to each corresponding array
        dance_array.append(features['danceability'])
        energy_array.append(features['energy'])
        key_array.append(features['key'])
        loudness_array.append(features['loudness'])
        mode_array.append(features['mode'])
        speech_array.append(features['speechiness'])
        acoustic_array.append(features['acousticness'])
        instrument_array.append(features['instrumentalness'])
        live_array.append(features['liveness'])
        valence_array.append(features['valence'])
        tempo_array.append(features['tempo'])
        link_array.append(song)
        
    #Put all song data in a library with proper labels
    song_data['track_name'] = name_array
    song_data['artists'] = artist_array
    song_data['popularity'] = popularity_array
    song_data['danceability'] = dance_array
    song_data['energy'] = energy_array
    song_data['key'] = key_array
    song_data['loudness'] = loudness_array
    song_data['mode'] = mode_array
    song_data['speechiness'] = speech_array
    song_data['acousticness'] = acoustic_array
    song_data['instrumentalness'] = instrument_array
    song_data['liveness'] = live_array
    song_data['valence'] = valence_array
    song_data['tempo'] = tempo_array
    song_data['link'] = link_array
    
    song_db = pd.DataFrame(song_data)
    return song_db

In [21]:
song_db = populate_song_info('https://open.spotify.com/playlist/7IKIjt9wZPPQWgvHo1SeFO?si=ee0a9c52dced4214')

In [22]:
song_db

,track_name,artists,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,link
0,No Stress,[Quincy],20,0.655,0.745,11,-6.068,0,0.2530,0.3670,0.000000,0.0959,0.593,89.874,https://open.spotify.com/track/7uXz2dQpT2FIRdJ...
1,Lucy,[Thutmose],28,0.687,0.669,5,-3.282,0,0.0356,0.2980,0.000000,0.1200,0.802,156.924,https://open.spotify.com/track/357Cpbxe8X6CK61...
2,Beg,"[Duckwrth, CLAY]",32,0.740,0.637,6,-7.517,0,0.1110,0.4200,0.000096,0.1070,0.734,129.906,https://open.spotify.com/track/0EjypH9hvvfx1a3...
3,ARRIVAL,[Matt Corman],18,0.746,0.736,10,-5.975,0,0.0826,0.1690,0.000104,0.4030,0.321,93.951,https://open.spotify.com/track/3BYDzLAuei0ZVeb...
4,HER.,[Sheperd],41,0.849,0.723,9,-5.142,0,0.2000,0.0198,0.003840,0.2690,0.590,108.084,https://open.spotify.com/track/0938VE4FbtsjbGA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,Esta Vida,"[Marshmello, Farruko]",82,0.755,0.676,5,-7.349,1,0.0377,0.0418,0.001760,0.2040,0.461,123.980,https://open.spotify.com/track/5OSGdSxtulzlzdP...
83,Every Good Girl,[Blxst],56,0.530,0.937,0,-2.757,1,0.3210,0.0457,0.000000,0.1510,0.460,92.916,https://open.spotify.com/track/6VYBCG9nCZwUUIm...
84,Popular (with Playboi Carti & Madonna) - From ...,"[The Weeknd, Playboi Carti, Madonna]",73,0.849,0.688,1,-6.340,1,0.1970,0.0435,0.000058,0.4120,0.829,99.013,https://open.spotify.com/track/38ziag7I4rieq2O...
85,Ic3,[Fuse ODG],34,0.847,0.778,6,-5.755,1,0.2280,0.2920,0.000000,0.3280,0.732,103.982,https://open.spotify.com/track/1FbBlINbLTXKZjL...


Have a Map to Show Where 

Going to have the option to either log in and save the playlist or just save it to excel

The flow should probably be as follows:

Part 1: Building the database
1. Extract songs from playlists
2. Perform extraction of all audio features
3. Need a way to append song name and audio feature info to the dataframe, OR perform a function call after the recommended songs are provided to get the song names and artist info

Part 2: Build the recommendations from cosine similarity

Part 3: Integration with Flask or Django

In [23]:
songlist = extract_songs('https://open.spotify.com/playlist/7IKIjt9wZPPQWgvHo1SeFO?si=ee0a9c52dced4214')

In [ ]:
df